## Imports

In [1]:
from __future__ import print_function

import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import collections
from tensorflow.keras.models import load_model

# From EHIL


import keras
from keras.layers.convolutional import ( Conv2D, MaxPooling2D, AveragePooling2D)
from keras.layers import (    Input,    Activation,    Dense,    Flatten)
from keras.layers import add
from keras.layers import LayerNormalization

from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.utils import np_utils

import os
import numpy as np

from sklearn.model_selection import train_test_split

2023-06-10 13:28:54.853707: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-10 13:28:54.904032: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 13:28:55.215712: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 13:28:55.216741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
summary_writer_h1 = tf.summary.create_file_writer('logs/H1')
summary_writer_h2 = tf.summary.create_file_writer('logs/H2')
summary_writer_h3 = tf.summary.create_file_writer('logs/H3')

## Config

In [ ]:
# Config
save_dir = os.path.join(os.getcwd(), 'Log')
Folder_Name = 'Videos_Database_20_Robot'

Type = 'WebCam'
##############################################################################

Process_Image_Flag = True
Robust_Flag = True

img_rows, img_cols = 256,256
img_rows, img_cols = 50,50

img_channels = 3
BATCH_SIZE = 20
nb_epoch = 100

Cup_Type = 'Big'
if Cup_Type == 'Medium':
    nb_classes = 9
if Cup_Type == 'Big':
    nb_classes = 10
if Cup_Type == 'Small':
    nb_classes = 7   



PHASE_CLASSES = 4
STATE_CLASSES = 10

## Load Data

In [ ]:


NPZ_Name = 'Data/Videos_Database_20_Robot_WebCam_50_overall_database.npz'
Database_Used = np.load(NPZ_Name)
Sessions = Database_Used['Session']
X_train = Database_Used['X_train']
Y_train_State = Database_Used['Y_train_State']
Y_train_Phase = Database_Used['Y_train_Context']
Y_train_Regress=Database_Used['Y_train_Regress']

In [ ]:
# create a list of the unique sessions
client_ids = np.unique(Sessions)

In [ ]:
def create_tf_dataset(client_ids, Database_used_col, categorical, categories):
  train_datasets = []
  test_datasets = []
  for session in client_ids:  
    # find the indices of the current session in the Sessions column of Database_Used
    session_indices = np.where(Sessions == session)[0]

    # get the X_train data for the current session
    session_X = Database_Used['X_train'][session_indices]
    # grab the training data for the necessary hierarchy
    session_Y = Database_Used[Database_used_col][session_indices]
    # if using categorical data, reshape the data for the model into one-hot encoded
    if categorical==True:
      session_Y = np_utils.to_categorical(session_Y, categories)
    # create train/test split
    X_train, X_test, Y_train, Y_test = train_test_split(session_X, session_Y, test_size=0.2, random_state=100)
    # Make into tf dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
    # Add to our list of datasets
    train_datasets.append([session,train_dataset])
    test_datasets.append([session,test_dataset])
  return train_datasets, test_datasets

In [ ]:
phase_train_datasets, phase_test_datasets = create_tf_dataset(client_ids, 'Y_train_Context', True, PHASE_CLASSES)
state_train_datasets, state_test_datasets = create_tf_dataset(client_ids, 'Y_train_State', True, STATE_CLASSES)
regress_train_datasets, regress_test_datasets = create_tf_dataset(client_ids, 'Y_train_Regress', False, None)

In [ ]:
def make_client_data(datasets):
  client_data = {}
  # loop through the datasets
  for dataset in datasets:
      # get the session name
      session = dataset[0]
      
      # get the session data
      session_data = dataset[1]
      
      # add the session data to the client_data dictionary
      client_data[session] = session_data
  return client_data

In [ ]:
# Make everything into a map for creating ClientData objects necessary for TFF federated learning

phase_train_client_data = make_client_data(phase_train_datasets)
phase_test_client_data = make_client_data(phase_test_datasets)
state_train_client_data = make_client_data(state_train_datasets)
state_test_client_data = make_client_data(state_test_datasets)
regress_train_client_data = make_client_data(regress_train_datasets)
regress_test_client_data = make_client_data(regress_test_datasets)

In [ ]:
client_ids = list(client_ids)

## Setup Federated Data

In [ ]:
def make_federated_data(client_data, client_ids):
  # Need a function to get the client data in order to make ClientData object
  def get_client_dataset(client_id):
    return client_data[client_id]

  # use tff to create ClientData object from our training data
  federated_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(client_ids, get_client_dataset)

  return federated_data

In [ ]:
phase_train_federated_data = make_federated_data(phase_train_client_data, client_ids)
phase_test_federated_data = make_federated_data(phase_test_client_data, client_ids)
state_train_federated_data = make_federated_data(state_train_client_data, client_ids)
state_test_federated_data = make_federated_data(state_test_client_data, client_ids)
regress_train_federated_data = make_federated_data(regress_train_client_data, client_ids)
regress_test_federated_data = make_federated_data(regress_test_client_data, client_ids)

In [ ]:
type(phase_train_federated_data)

tensorflow_federated.python.simulation.datasets.client_data.ConcreteClientData

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset, num_classes):
    def batch_format_fn(image, label):
        """Prepare a batch of data and return a (features, label) tuple."""
        batch_size = tf.shape(image)[0]  # Get the current batch size
        return (tf.reshape(image, [batch_size, 50, 50, 3]),
                tf.reshape(label, [batch_size, num_classes]))

    return dataset.batch(BATCH_SIZE).map(batch_format_fn)

def preprocess_federated_data(federated_data, num_classes):
  client_ids = sorted(federated_data.client_ids)[:NUM_CLIENTS]
  federated_data = [preprocess(federated_data.create_tf_dataset_for_client(x), num_classes)
                          for x in client_ids]
  return federated_data



In [ ]:
phase_train = preprocess_federated_data(phase_train_federated_data, PHASE_CLASSES)
state_train = preprocess_federated_data(state_train_federated_data, STATE_CLASSES)
regress_train = preprocess_federated_data(regress_train_federated_data, 1)

In [ ]:
state_train[0].element_spec

(TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))

## Setup EHIL Models

### Functions

In [ ]:
###############################################################################
'''
Functions
'''
###############################################################################

from keras.initializers import glorot_uniform

def lr_schedule(epoch):
    '''
    epoch: number of epochs for model training
    lr: learning rate
    ''' 
    lr = 1e-3
    if epoch > 160:
        lr *= 0.5e-3
    elif epoch > 120:
        lr *= 1e-3
    elif epoch > 80:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



def Conv_bn_relu(infor, **conv_params):
    '''
    Build conv -> BN -> relu block
    '''
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))


    conv = Conv2D(filters=filters, kernel_size=kernel_size,
                  strides=strides, padding=padding,
                  kernel_initializer=kernel_initializer,
                  kernel_regularizer=kernel_regularizer)(infor)
    
    
    norm = LayerNormalization(axis=CHANNEL_AXIS)(conv)
    out = Activation("relu")(norm)

    return out 


#Reference: http://arxiv.org/pdf/1603.05027v2.pdf
def Bn_relu_conv(infor,**conv_params):
    '''
    Build a BN -> relu -> conv block.
    '''
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    norm = LayerNormalization(axis=CHANNEL_AXIS)(infor)

    activation = Activation("relu")(norm)
        
    out = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return out



def basic_block(BlockIn, filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    '''
    Basic 3 X 3 convolution blocks
    '''

    if is_first_block_of_first_layer:
        conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                       strides=init_strides,
                       padding="same",
                       kernel_initializer="he_normal",
                       kernel_regularizer=l2(1e-4))(BlockIn)
    else:
        conv1 = Bn_relu_conv(infor = BlockIn,filters=filters, kernel_size=(3, 3),
                              strides=init_strides)

    residual = Bn_relu_conv(infor = conv1,filters=filters, kernel_size=(3, 3))


    input_shape = K.int_shape(BlockIn)
    residual_shape = K.int_shape(residual)
    
    # stride should be set properly and match  (width, height) of residual
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    
    
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(BlockIn)        
    else:
        shortcut = BlockIn
    
    # Adds a shortcut between input and residual block
    return add([shortcut, residual])


def buildmodel(input_shape, num_outputs, Regress_Flag):
        '''

        input_shape: (nb_channels, nb_rows, nb_cols)
        num_outputs:  number of outputs at final softmax layer
        Regress_Flag: classify or regress

        '''

        global ROW_AXIS
        global COL_AXIS
        global CHANNEL_AXIS
        
        # if K.image_data_format() == 'channels_last':
        
        #     ROW_AXIS = 1; COL_AXIS = 2;  CHANNEL_AXIS = 3
        # else:
        #     CHANNEL_AXIS = 1; ROW_AXIS = 2; COL_AXIS = 3        

        if tf.keras.backend.image_data_format() == 'channels_last':
            ROW_AXIS = 1
            COL_AXIS = 2
            CHANNEL_AXIS = 3
        else:
            CHANNEL_AXIS = 1
            ROW_AXIS = 2
            COL_AXIS = 3

        
        
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_data_format() == 'channels_last':
        #if K.image_dim_ordering() == 'tf':

            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        input = Input(shape=input_shape, name='main_input')
        conv1 = Conv_bn_relu(input, filters=64, kernel_size=(7, 7), strides=(2, 2))
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same", name = "conv_pool1")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate([2,2,2,2]):
            '''
            residual block with repeating bottleneck blocks
            '''

            is_first_layer=(i == 0)
            for i in range(r):
                init_strides = (1, 1)
                if i == 0 and not is_first_layer:
                    init_strides = (2, 2)
                block = basic_block(BlockIn = block, filters=filters, init_strides=init_strides,
                                       is_first_block_of_first_layer=(is_first_layer and i == 0))
                
            filters *= 2

        # Last activation
    
        norm = LayerNormalization(axis=CHANNEL_AXIS)(block)
        block = Activation("relu")(norm)

        # classifier block
        block_shape = K.int_shape(block)
        pool2_out = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1), name = "conv_final")(block)
        
        #flatten1 = keras.layers.GlobalAveragePooling2D(name = "GAP")(pool2)
        #out = keras.layers.Dense(num_outputs,activation='softmax')(pooled)

        flatten1 = Flatten( name = "Flatten")(pool2_out)

        flatten1 = Dense(128, activation='relu',name = "Flatten2")(flatten1)

        if Regress_Flag == False:
            
            dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax", name = "Dense_layer")(flatten1)
        else:
            dense = Dense(units=1, activation='linear', kernel_initializer=glorot_uniform(seed=0))(flatten1)
            
            #dense = Dense(units=1, kernel_initializer="he_normal", activation="linear", name = "Dense_layer")(flatten1)
  
        model = Model(inputs=input, outputs=dense)
        return model


def learn_model(oldmodel,nb_classes,Transfer_Type,summary=False): 

    base_model = oldmodel
    
    if Transfer_Type == 'Classification':
        intermediate_layer_model = Model(inputs=base_model.input,outputs=base_model.get_layer("conv_final").output)

    if Transfer_Type == 'Regression':
        #intermediate_layer_model = Model(inputs=base_model.input,outputs=base_model.get_layer("Flatten1").output)
        intermediate_layer_model = Model(inputs=base_model.input,outputs=base_model.get_layer("Dense_Classification").output)
    
    x = intermediate_layer_model(base_model.input)
    if nb_classes >1:
        x = keras.layers.GlobalAveragePooling2D()(x)# 添加全局平均池化层 
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    
    
    if Transfer_Type == 'Classification':
        dense = Dense(units=nb_classes, kernel_initializer="he_normal",
                  activation="softmax", name = "Dense_Classification")(x)
    if Transfer_Type == 'Regression':
        dense = Dense(units=1, activation='linear', name = "Dense_Regression", kernel_initializer=glorot_uniform(seed=0))(x)

    model = Model(inputs=base_model.input, outputs=dense)
    
    
    # show summary if specified
    if summary==True : 
        model.summary()

    if Transfer_Type == 'Classification':
        # choose the optimizer 
        #optimizer = keras.optimizers.Adam()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
    if Transfer_Type == 'Regression':
        model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['mse'])

    return model
 

### H1

In [ ]:
'''
Prepare to train model H1
'''
# nb_classes = 4 
# Model_Type = 'Phase'

def build_h1_model():
  return buildmodel((img_channels, img_rows, img_cols), PHASE_CLASSES,False)

h1_model = build_h1_model()
h1_example = build_h1_model()

BUILD_FN = build_h1_model
INPUT_SPEC = phase_train[0].element_spec

### H2

In [ ]:
h2_template = learn_model(h1_example, STATE_CLASSES, 'Classification',False)

def build_h2_model():
  return tf.keras.models.clone_model(h2_template)
  # return buildmodel((img_channels, img_rows, img_cols), STATE_CLASSES,False)

h2_example = build_h2_model()


### H3

In [ ]:
h3_template = learn_model(h2_example, 1, 'Regression',False)

def build_h3_model():
  return tf.keras.models.clone_model(h3_template)

## Setup Federated Learning

### H1

In [ ]:
def model_fn_h1():
  model = build_h1_model()
  return tff.learning.models.from_keras_model(
      model,
      input_spec=phase_train[0].element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.Accuracy()])

In [ ]:
@tff.tf_computation
def server_init_h1():
  model = model_fn_h1()
  return model.trainable_variables

In [ ]:
@tff.federated_computation
def initialize_fn_h1():
  return tff.federated_value(server_init_h1(), tff.SERVER)

In [ ]:
@tf.function
def client_update_h1(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [ ]:
@tf.function
def server_update_h1(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [ ]:
h1_fed = model_fn_h1()
tf_dataset_type_h1 = tff.SequenceType(h1_fed.input_spec)

In [ ]:
str(tf_dataset_type_h1)

'<float32[?,50,50,3],float32[?,4]>*'

In [ ]:
model_weights_type_h1 = h1_fed.trainable_variables
# Assuming model_weights_type is a list of trainable variables
model_weights_type_h1 = [v for v in model_weights_type_h1]

model_weights_type_h1 = tff.to_type([tf.TensorSpec.from_tensor(v.value()) for v in model_weights_type_h1])

In [ ]:
@tff.tf_computation(tf_dataset_type_h1, model_weights_type_h1)
def client_update_fn_h1(tf_dataset, server_weights):
  model = model_fn_h1()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update_h1(model, tf_dataset, server_weights, client_optimizer)

2023-06-10 12:57:08.936132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_5' with dtype float and shape [?,1,1,512]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_5}}]]
2023-06-10 12:57:08.936292: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_6' with dtype float and shape [?,2,2,512]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_6}}]]
2023-06-10 12:57:08.936342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG IN

In [ ]:
@tff.tf_computation(model_weights_type_h1)
def server_update_fn_h1(mean_client_weights):
  model = model_fn_h1()
  return server_update_h1(model, mean_client_weights)

In [ ]:
federated_server_type_h1 = tff.FederatedType(model_weights_type_h1, tff.SERVER)
federated_dataset_type_h1 = tff.FederatedType(tf_dataset_type_h1, tff.CLIENTS)

In [ ]:
@tff.federated_computation(federated_server_type_h1, federated_dataset_type_h1)
def next_fn_h1(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn_h1, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn_h1, mean_client_weights)


  return server_weights

In [ ]:
federated_algorithm_h1 = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn_h1,
    next_fn=next_fn_h1
)

### H2

In [ ]:
def model_fn_h2():
  model = build_h2_model()
  return tff.learning.models.from_keras_model(
      model,
      input_spec=state_train[0].element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.Accuracy()])

In [ ]:
@tff.tf_computation
def server_init_h2():
  model = model_fn_h2()
  return model.trainable_variables

In [ ]:
@tff.federated_computation
def initialize_fn_h2():
  return tff.federated_value(server_init_h2(), tff.SERVER)

In [ ]:
@tf.function
def client_update_h2(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [ ]:
@tf.function
def server_update_h2(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [ ]:
h2_fed = model_fn_h2()
tf_dataset_type_h2 = tff.SequenceType(h2_fed.input_spec)

In [ ]:
str(tf_dataset_type_h2)

'<float32[?,50,50,3],float32[?,10]>*'

In [ ]:
model_weights_type_h2 = h2_fed.trainable_variables
# Assuming model_weights_type is a list of trainable variables
model_weights_type_h2 = [v for v in model_weights_type_h2]

model_weights_type_h2 = tff.to_type([tf.TensorSpec.from_tensor(v.value()) for v in model_weights_type_h2])

In [ ]:
@tff.tf_computation(tf_dataset_type_h2, model_weights_type_h2)
def client_update_fn_h2(tf_dataset, server_weights):
  model = model_fn_h2()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update_h2(model, tf_dataset, server_weights, client_optimizer)

2023-06-10 12:57:16.285322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_7' with dtype float and shape [?,1,1,512]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_7}}]]
2023-06-10 12:57:16.285719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_8' with dtype int32 and shape [2]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_8}}]]
2023-06-10 12:57:16.285757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Exec

In [ ]:
@tff.tf_computation(model_weights_type_h2)
def server_update_fn_h2(mean_client_weights):
  model = model_fn_h2()
  return server_update_h2(model, mean_client_weights)

In [ ]:
federated_server_type_h2 = tff.FederatedType(model_weights_type_h2, tff.SERVER)
federated_dataset_type_h2 = tff.FederatedType(tf_dataset_type_h2, tff.CLIENTS)

In [ ]:
@tff.federated_computation(federated_server_type_h2, federated_dataset_type_h2)
def next_fn_h2(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn_h2, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn_h2, mean_client_weights)

  return server_weights

In [ ]:
federated_algorithm_h2 = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn_h2,
    next_fn=next_fn_h2
)

### H3

In [ ]:
def model_fn_h3():
  model = build_h3_model()
  return tff.learning.models.from_keras_model(
      model,
      input_spec=regress_train[0].element_spec,
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.MeanSquaredError()])

In [ ]:
@tff.tf_computation
def server_init_h3():
  model = model_fn_h3()
  return model.trainable_variables

In [ ]:
@tff.federated_computation
def initialize_fn_h3():
  return tff.federated_value(server_init_h3(), tff.SERVER)

In [ ]:
@tf.function
def client_update_h3(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [ ]:
@tf.function
def server_update_h3(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [ ]:
h3_fed = model_fn_h3()
tf_dataset_type_h3 = tff.SequenceType(h3_fed.input_spec)

In [ ]:
str(tf_dataset_type_h3)

'<float32[?,50,50,3],float64[?,1]>*'

In [ ]:
model_weights_type_h3 = h3_fed.trainable_variables
# Assuming model_weights_type is a list of trainable variables
model_weights_type_h3 = [v for v in model_weights_type_h3]

model_weights_type_h3 = tff.to_type([tf.TensorSpec.from_tensor(v.value()) for v in model_weights_type_h3])

In [ ]:
@tff.tf_computation(tf_dataset_type_h3, model_weights_type_h3)
def client_update_fn_h3(tf_dataset, server_weights):
  model = model_fn_h3()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update_h3(model, tf_dataset, server_weights, client_optimizer)

2023-06-10 12:57:24.632060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_12' with dtype float and shape [?,1,1,512]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_12}}]]
2023-06-10 12:57:24.632181: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_13' with dtype int32 and shape [2]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_13}}]]
2023-06-10 12:57:24.632224: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) 

In [ ]:
@tff.tf_computation(model_weights_type_h3)
def server_update_fn_h3(mean_client_weights):
  model = model_fn_h3()
  return server_update_h3(model, mean_client_weights)

In [ ]:
federated_server_type_h3 = tff.FederatedType(model_weights_type_h3, tff.SERVER)
federated_dataset_type_h3 = tff.FederatedType(tf_dataset_type_h3, tff.CLIENTS)

In [ ]:
@tff.federated_computation(federated_server_type_h3, federated_dataset_type_h3)
def next_fn_h3(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn_h3, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn_h3, mean_client_weights)

  return server_weights

In [ ]:
federated_algorithm_h3 = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn_h3,
    next_fn=next_fn_h3
)

## Model Evaluation

### Creating test sets

In [ ]:
def create_test_set(federated_test_set, num_classes):
  # Create a list to store client datasets
  client_datasets = []

  # Iterate over client IDs and create datasets
  for client_id in federated_test_set.client_ids:
      client_dataset = federated_test_set.create_tf_dataset_for_client(client_id)
      client_datasets.append(client_dataset)

  # Combine the client datasets into a centralized dataset
  test_set = tf.data.experimental.sample_from_datasets(client_datasets)
  test_set = preprocess(test_set, num_classes)
  return test_set

In [ ]:
phase_test_central = create_test_set(phase_test_federated_data, PHASE_CLASSES)
state_test_central = create_test_set(state_test_federated_data, STATE_CLASSES)
regress_test_central = create_test_set(regress_test_federated_data, 1)

Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.


Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.


In [ ]:
type(phase_test_central)

tensorflow.python.data.ops.map_op._MapDataset

### Evaluation functions

In [ ]:
# Model compile instructions taken from Dandan's code
def evaluate_h1(server_state):
  model = build_h1_model()
  model.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy']  
  )
  model.set_weights(server_state)
  model.evaluate(phase_test_central)

In [ ]:
# Model compile instructions taken from Dandan's code
def evaluate_h2(server_state):
  model = build_h2_model()
  model.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy']  
  )
  model.set_weights(server_state)
  model.evaluate(state_test_central)

In [ ]:
# Model compile instructions taken from Dandan's code
def evaluate_h3(server_state):
  model = build_h3_model()
  model.compile(
      loss='mean_squared_error',
      optimizer='adam',
      metrics=['mse']  
  )
  model.set_weights(server_state)
  model.evaluate(regress_test_central)

### H1

In [ ]:
server_state_h1 = federated_algorithm_h1.initialize()
evaluate_h1(server_state_h1)

2023-06-10 12:57:28.280992: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 12:57:28.281234: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 12:57:31.375481: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype float and shape [42,50,50,3]
	 [[{{node Placeholder/_7}}]]
2023-06-10 12:57:31.376430: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_41' with dtype float and shape [50,4]
	 [[{{node Placeholder/_41}}]]


57/57 [==============================] - 5s 64ms/step - loss: 2.0125 - accuracy: 0.2246


### H2

In [ ]:
server_state_h2 = federated_algorithm_h2.initialize()
evaluate_h2(server_state_h2)

2023-06-10 12:57:36.940932: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 12:57:36.941081: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 12:57:38.377607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_22' with dtype float and shape [44,50,50,3]
	 [[{{node Placeholder/_22}}]]
2023-06-10 12:57:38.378515: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_70' with dtype float and shape [39,50,50,3]
	 [[{{node Placeholder/_70}}]]


57/57 [==============================] - 5s 63ms/step - loss: 3.5214 - accuracy: 0.0693


### H3

In [ ]:
server_state_h3 = federated_algorithm_h3.initialize()
evaluate_h3(server_state_h3)

2023-06-10 12:57:43.691192: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 12:57:43.691321: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 12:57:45.695806: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_53' with dtype double and shape [53,1]
	 [[{{node Placeholder/_53}}]]
2023-06-10 12:57:45.696758: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_62' with dtype double and shape [53,1]
	 [[{{node Placeholder/_62}}]]


57/57 [==============================] - 5s 63ms/step - loss: 1.1061 - mse: 0.1464


## Build Eval Models (TFF glitch)

In [ ]:
h1_eval = build_h1_model()
h1_eval.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy']  
  ) 

In [ ]:
h2_eval = build_h2_model()
h2_eval.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy']  
  )

In [ ]:
h3_eval = build_h3_model()
h3_eval.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mse']  
)

## Model Training

### H1

In [ ]:
with summary_writer_h1.as_default():
    for round in range(5):
        server_state_h1 = federated_algorithm_h1.next(server_state_h1, phase_train)
        # evaluation - run quietly
        h1_eval.set_weights(server_state_h1.model)
        loss, accuracy = h1_eval.evaluate(phase_test_central, verbose=0)
        print('round {:2d}, loss={:.3f}, accuracy={:.3f}'.format(round, loss, accuracy))
        # save the metrics
        tf.summary.scalar('loss', loss, step=round)
        tf.summary.scalar('accuracy', accuracy, step=round)

### H2

In [ ]:
with summary_writer_h2.as_default():
    for round in range(5):
        server_state_h2 = federated_algorithm_h2.next(server_state_h2, state_train)
        # evaluation
        h2_eval.set_weights(server_state_h2)  
        loss, accuracy = h2_eval.evaluate(state_test_central, verbose=0)
        print('round {:2d}, loss={:.3f}, accuracy={:.3f}'.format(round, loss, accuracy))
        # save the metrics
        tf.summary.scalar('loss', loss, step=round)
        tf.summary.scalar('accuracy', accuracy, step=round)

2023-06-10 13:05:39.890113: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 13:05:40.850377: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 13:05:41.868494: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 13:05:41.868681: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 13:05:41.990673: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 13:05:41.990917: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 13:05:42.237388: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 13:05:42.237603: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


### H3

In [ ]:
with summary_writer_h3.as_default():
    for round in range(5):
        server_state_h3 = federated_algorithm_h3.next(server_state_h3, regress_train)
        # evaluation
        h3_eval.set_weights(server_state_h3)  
        loss, accuracy = h3_eval.evaluate(regress_test_central, verbose=0)
        print('round {:2d}, loss={:.3f}, accuracy={:.3f}'.format(round, loss, accuracy))
        # save the metrics
        tf.summary.scalar('loss', loss, step=round)
        tf.summary.scalar('accuracy', accuracy, step=round)

2023-06-09 23:11:27.359409: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-09 23:11:27.373792: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-09 23:11:29.262573: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-09 23:11:29.262833: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-09 23:11:29.565037: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-09 23:11:29.565267: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-09 23:11:30.042798: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-09 23:11:30.042985: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


## Post-Training evaluation

### H1

In [ ]:
h1_eval.set_weights(server_state_h1)
h1_eval.evaluate(phase_test_central)

57/57 [==============================] - 17s 140ms/step - loss: 1.2908 - accuracy: 0.9254


[1.290783405303955, 0.9254385828971863]

In [ ]:
h1_eval.save('Models/h1_eval.h5')

### H2

In [ ]:
h2_eval.set_weights(server_state_h2)
h2_eval.evaluate(state_test_central)

57/57 [==============================] - 10s 95ms/step - loss: 1.6620 - accuracy: 0.7798


[1.662004828453064, 0.7798245549201965]

In [ ]:
h2_eval.save('Models/h2_eval.h5')

### H3

In [ ]:
h3_eval.set_weights(server_state_h3)
h3_eval.evaluate(regress_test_central)

57/57 [==============================] - 9s 98ms/step - loss: 1.0638 - mse: 0.1069


[1.0638397932052612, 0.10685588419437408]

In [ ]:
h3_eval.save('Models/h3_eval.h5')

## Tensorboard Visualizations

In [ ]:
!ls {'Logs/H1/'}
!tensorboard --logdir {'Logs/H1/'} --port=0

In [ ]:
!ls {'Logs/H2/'}
!tensorboard --logdir {'Logs/H2/'} --port=0

In [ ]:
!ls {'Logs/H3/'}
!tensorboard --logdir {'Logs/H3/'} --port=0